In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [4]:
# Section 1 : Données Vectorielles
# Créer une carte interactive pour les données vectorielles
Map_vector = geemap.Map()

# Importer les pays (0-3 levels shapefiles)
countries = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm0_anat_20240520')
regions = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm1_anat_20240520')
districts = ee.FeatureCollection('projects/ee-papaamadouniang2004/assets/sen_admbnda_adm2_anat_20240520')

# Calculs statistiques : nombre de géométries, CRS, extent, projection
country_count = countries.size().getInfo()
print(f'Nombre de géométries (pays) : {country_count}')

crs_country = countries.first().geometry().projection().getInfo()
print(f'CRS du shapefile des pays : {crs_country}')

extent_country = countries.geometry().bounds().getInfo()
print(f'Extent (limites) des pays : {extent_country}')

# Calculs statistiques 2 : centroïdes, aire, périmètre
centroids = countries.map(lambda feature: feature.setGeometry(feature.geometry().centroid()))
country_area = regions.geometry().area().getInfo()
print(f'Aire totale du pays (en mètres carrés) : {country_area}')

country_perimeter = countries.geometry().perimeter().getInfo()
print(f'Périmètre total du pays (en mètres) : {country_perimeter}')

# Visualisation
Map_vector.addLayer(countries, {}, 'Pays')
Map_vector.addLayer(centroids, {'color': 'red'}, 'Centroïdes des pays')
Map_vector.centerObject(regions, 3)

# Afficher la carte
Map_vector

Nombre de géométries (pays) : 1
CRS du shapefile des pays : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Extent (limites) des pays : {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-17.53114756482535, 12.308126642857681], [-11.345215008418714, 12.308126642857681], [-11.345215008418714, 16.69280533655572], [-17.53114756482535, 16.69280533655572], [-17.53114756482535, 12.308126642857681]]]}
Aire totale des pays (en mètres carrés) : 197514802862.19406
Périmètre total des pays (en mètres) : 3441351.1148358406


Map(center=[14.358557193311563, -14.466239286231914], controls=(WidgetControl(options=['position', 'transparen…

In [3]:
# Section 2 : Données Raster
# Créer une carte interactive pour les données raster
Map_raster = geemap.Map()

# Importer les rasters (5)
raster1 = ee.Image('projects/ee-papaamadouniang2004/assets/202406_Global_Pf_Parasite_Rate_SEN_2022')
raster2 = ee.Image('projects/ee-papaamadouniang2004/assets/202406_Global_Pf_Parasite_Rate_SEN_2021')
raster3 = ee.Image('projects/ee-papaamadouniang2004/assets/202406_Global_Pf_Parasite_Rate_SEN_2020')
raster4 = ee.Image('projects/ee-papaamadouniang2004/assets/202406_Global_Pf_Parasite_Rate_SEN_2019')
raster5 = ee.Image('projects/ee-papaamadouniang2004/assets/202406_Global_Pf_Parasite_Rate_SEN_2018')

# Liste des rasters
rasters = [raster1, raster2, raster3, raster4, raster5]

# Vérifier la taille des pixels pour déterminer la résolution la plus fine
pixel_sizes = [raster.projection().nominalScale().getInfo() for raster in rasters]
min_pixel_size = min(pixel_sizes)
print(f'Tailles des pixels des rasters : {pixel_sizes}')
print(f'Taille de pixel la plus fine : {min_pixel_size}')

# Reprojeter tous les rasters à la résolution la plus fine
rasters_resampled = [raster.reproject(raster.projection(), None, min_pixel_size) for raster in rasters]

# Calculer les statistiques : médiane, moyenne, écart type, min, max et créer une image pour chaque statistique
# Combiner tous les rasters en une seule image avec plusieurs bandes pour chaque statistique
median_image = ee.ImageCollection(rasters_resampled).reduce(ee.Reducer.median())
mean_image = ee.ImageCollection(rasters_resampled).reduce(ee.Reducer.mean())
std_dev_image = ee.ImageCollection(rasters_resampled).reduce(ee.Reducer.stdDev())
min_image = ee.ImageCollection(rasters_resampled).reduce(ee.Reducer.min())
max_image = ee.ImageCollection(rasters_resampled).reduce(ee.Reducer.max())

# Afficher les statistiques des images raster
median_stats = median_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=countries.geometry(), scale=min_pixel_size).getInfo()
mean_stats = mean_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=countries.geometry(), scale=min_pixel_size).getInfo()
std_dev_stats = std_dev_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=countries.geometry(), scale=min_pixel_size).getInfo()
min_stats = min_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=countries.geometry(), scale=min_pixel_size).getInfo()
max_stats = max_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=countries.geometry(), scale=min_pixel_size).getInfo()

print(f'Statistiques des images raster :')
print(f' - Médiane : {median_stats}')
print(f' - Moyenne : {mean_stats}')
print(f' - Écart type : {std_dev_stats}')
print(f' - Min : {min_stats}')
print(f' - Max : {max_stats}')

# Visualiser les images des statistiques
Map_raster.addLayer(median_image, {}, 'Médiane des Rasters')
Map_raster.addLayer(mean_image, {}, 'Moyenne des Rasters')
Map_raster.addLayer(std_dev_image, {}, 'Écart Type des Rasters')
Map_raster.addLayer(min_image, {}, 'Min des Rasters')
Map_raster.addLayer(max_image, {}, 'Max des Rasters')

# Afficher la carte
Map_raster

Tailles des pixels des rasters : [4637.422865601593, 4637.422865601593, 4637.422865601593, 4637.422865601593, 4637.422865601593]
Taille de pixel la plus fine : 4637.422865601593
Statistiques des images raster :
 - Médiane : {'b1_median': 0.03865163523397902, 'b2_median': None}
 - Moyenne : {'b1_mean': 0.039306175054777806, 'b2_mean': None}
 - Écart type : {'b1_stdDev': 0.009551975310538412, 'b2_stdDev': None}
 - Min : {'b1_min': 0.027236734600569988, 'b2_min': None}
 - Max : {'b1_max': 0.055179217951073556, 'b2_max': None}


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…